In [1]:
!pip install /kaggle/input/rsna-2022-whl/{pydicom-2.3.0-py3-none-any.whl,pylibjpeg-1.4.0-py3-none-any.whl,python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl}
!pip install /kaggle/input/rsna-2022-whl/{torch-1.12.1-cp37-cp37m-manylinux1_x86_64.whl,torchvision-0.13.1-cp37-cp37m-manylinux1_x86_64.whl}

Processing /kaggle/input/rsna-2022-whl/pydicom-2.3.0-py3-none-any.whl
Processing /kaggle/input/rsna-2022-whl/pylibjpeg-1.4.0-py3-none-any.whl
Processing /kaggle/input/rsna-2022-whl/python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
pydicom is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/rsna-2022-whl/torch-1.12.1-cp37-cp37m-manylinux1_x86_64.whl
Processing /kaggle/input/rsna-2022-whl/torchvision-0.13.1-cp37-cp37m-manylinux1_x86_64.whl
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0
    Uninstalling torchvision-0.12.0:
      Successfully uninstalled torchvision-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed

In [2]:
!pip install /kaggle/input/rsna-weights/timm-0.5.4-py3-none-any.whl
!pip install /kaggle/input/rsna-weights/tifffile-2022.8.8-py3-none-any.whl
!pip install /kaggle/input/rsna-weights/einops-0.5.0-py3-none-any.whl



Processing /kaggle/input/rsna-weights/timm-0.5.4-py3-none-any.whl
Processing /kaggle/input/rsna-weights/tifffile-2022.8.8-py3-none-any.whl
ERROR: Package 'tifffile' requires a different Python: 3.7.12 not in '>=3.8'
Processing /kaggle/input/rsna-weights/einops-0.5.0-py3-none-any.whl


In [3]:
import os
os.listdir('/kaggle/input/mmdetection-2-17-offline')

!pip install /kaggle/input/mmdetection-2-17-offline/mmcv_full-1.3.14-cp37-cp37m-linux_x86_64.whl --no-deps
!pip install /kaggle/input/mmdetection-2-17-offline/pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl --no-deps
!pip install /kaggle/input/mmdetection-2-17-offline/terminaltables-3.1.0-py3-none-any.whl --no-deps
!pip install /kaggle/input/mmdetection-2-17-offline/pytest_runner-5.3.1-py3-none-any.whl --no-deps
!pip install /kaggle/input/mmdetection-2-17-offline/mmpycocotools-12.0.3-cp37-cp37m-linux_x86_64.whl --no-deps
!pip install /kaggle/input/mmdetection-2-17-offline/terminal-0.4.0-py3-none-any.whl --no-deps
!pip install /kaggle/input/mmdetection-2-17-offline/mmdet-2.17.0-py3-none-any.whl --no-deps
!pip install /kaggle/input/mmdetection-2-17-offline/addict-2.4.0-py3-none-any.whl --no-deps
!pip install /kaggle/input/mmdetection-2-17-offline/yapf-0.31.0-py2.py3-none-any.whl --no-deps

Processing /kaggle/input/mmdetection-2-17-offline/mmcv_full-1.3.14-cp37-cp37m-linux_x86_64.whl
Processing /kaggle/input/mmdetection-2-17-offline/pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl
Processing /kaggle/input/mmdetection-2-17-offline/terminaltables-3.1.0-py3-none-any.whl
Processing /kaggle/input/mmdetection-2-17-offline/pytest_runner-5.3.1-py3-none-any.whl
Processing /kaggle/input/mmdetection-2-17-offline/mmpycocotools-12.0.3-cp37-cp37m-linux_x86_64.whl
Processing /kaggle/input/mmdetection-2-17-offline/terminal-0.4.0-py3-none-any.whl
Processing /kaggle/input/mmdetection-2-17-offline/mmdet-2.17.0-py3-none-any.whl
Processing /kaggle/input/mmdetection-2-17-offline/addict-2.4.0-py3-none-any.whl
Processing /kaggle/input/mmdetection-2-17-offline/yapf-0.31.0-py2.py3-none-any.whl
  Attempting uninstall: yapf
    Found existing installation: yapf 0.32.0
    Uninstalling yapf-0.32.0:
      Successfully uninstalled yapf-0.32.0


In [4]:
import sys
sys.path.append('/kaggle/input/rsnazoopublic')


In [5]:
import os
import random
import re
from dataclasses import dataclass
from typing import Dict
from typing import List

import albumentations
import cv2
import numpy as np
import pydicom
import tifffile
import torch
import torch.hub
from albumentations import ReplayCompose
from skimage import measure
from torch.functional import Tensor
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [6]:

@dataclass
class BatchSlice:
    i_from: int
    i_to: int
    i_start: int


def get_slices(batch: Tensor, dim=1, window: int = 16, overlap: int = 8) -> List[BatchSlice]:
    num_imgs = batch.size(dim)
    if num_imgs <= window:
        return [BatchSlice(0, num_imgs, 0)]
    stride = window - overlap
    result = []
    current_idx = 0
    while True:
        next_idx = current_idx + window

        if next_idx >= num_imgs:
            current_idx = num_imgs - window
            offset = overlap // 2 if current_idx > 0 else 0
            next_idx = num_imgs
            result.append(BatchSlice(current_idx, next_idx, offset))
            break
        else:
            offset = overlap // 2 if current_idx > 0 else 0
            result.append(BatchSlice(current_idx, next_idx, offset))
        current_idx += stride
    return result

In [7]:
def combine_scan(scan_dir: str, size=512, fix_monochrome: bool = True) -> np.ndarray:
    num_files = len(os.listdir(scan_dir))
    images = []
    offset = 0
    first = None
    last = None
    files = []
    for i in range(num_files):
        dpath = os.path.join(scan_dir, f"{i + offset}.dcm")
        if i == 0:
            while not os.path.exists(dpath):
                offset += 1
                dpath = os.path.join(scan_dir, f"{i + offset}.dcm")
        files.append(dpath)

    for dpath in files[::2]:
        ds = pydicom.dcmread(dpath)
        if not first:
            first = ds
        last = ds

        data = ds.pixel_array
        data = cv2.resize(data, (size, size))
        if fix_monochrome and ds.PhotometricInterpretation == "MONOCHROME1":
            data = np.amax(data) - data
        images.append(data)

    if first and last:
        if last.ImagePositionPatient[2] > first.ImagePositionPatient[2]:
            images = images[::-1]
    return np.array(images)

class DatasetSeg(Dataset):
    def __init__(
            self,
            dataset_dir: str,
            cases: List[str],
    ):
        self.dataset_dir = dataset_dir
        self.cases = cases

    def __getitem__(self, i):
        cube_id = self.cases[i]
        image_cube = combine_scan(os.path.join(self.dataset_dir, cube_id), size=256)
        image_mean = image_cube.mean()
        image_std = image_cube.std()
        h = image_cube.shape[0]

        images = image_cube
        if h % 32 > 0:
            tmp = np.zeros(((h // 32 + 1) * 32, 256, 256))
            tmp[:h] = images
            images = tmp
        images = (images - image_mean) / image_std
        images = np.expand_dims(images, 0)
        sample = {}
        sample['image'] = torch.from_numpy(images).float()
        sample['cube_id'] = cube_id
        sample['h'] = h
        return sample

    def __len__(self):
        return len(self.cases)


crop_augs =  albumentations.ReplayCompose([
            albumentations.LongestMaxSize(256),
            albumentations.PadIfNeeded(256, 256, border_mode=cv2.BORDER_CONSTANT),
        ])

class DatasetCrops(Dataset):
    def __init__(
            self,
            dataset_dir: str,
            cases: List[str],
            transforms=crop_augs,
            slice_size=40,
    ):
        self.dataset_dir = dataset_dir
        self.transforms = transforms
        self.slice_size = slice_size
        self.cases = cases

    def __getitem__(self, i):
        cube_id = self.cases[i]
        mask_cube = tifffile.imread(os.path.join("seg_preds", f"{cube_id}.tif"))
        image_cube = combine_scan(os.path.join(self.dataset_dir, cube_id) ,size=512)
        boxes = {}
        for rprop in measure.regionprops(mask_cube):
            boxes[rprop.label] = rprop.bbox, rprop.area

        image_mean = image_cube.mean()
        image_std = image_cube.std()
        slice_size = self.slice_size
        all_images = []
        labels = np.zeros((8,))
        for li in range(1, 8):
            if li not in boxes:
                all_images.append(np.zeros((3, self.slice_size, 256, 256)))
            else:
                bbox, area = boxes[li]
                z1, z2 = bbox[0], bbox[3]
                y1, y2 = max(bbox[1] - 16, 0), min(bbox[4] + 16, 256)
                x1, x2 = max(bbox[2] - 16, 0), min(bbox[5] + 16, 256)
                # if z2 - z1 < slice_size:
                #     z1 = random.randint(max(z2 - slice_size, 0), z1)
                #     z2 = z1 + slice_size
                # todo: verify
                if z2 - z1 < slice_size:
                    diff = (slice_size - z2 + z1) // 2
                    z1 = max(0, z1 - diff)
                    z2 = z1 + slice_size
                images = image_cube[z1:z2, y1 * 2:y2 * 2, x1 * 2:x2 * 2].copy()
                masks = mask_cube[z1:z2, y1:y2, x1:x2].copy()
                slice_size = self.slice_size

                replay = None
                image_crops = []
                mask_crops = []
                for i in range(images.shape[0]):
                    image = images[i]
                    mask = masks[i]
                    h, w, = mask.shape
                    mask = cv2.resize(mask, (w * 2, h * 2), interpolation=cv2.INTER_NEAREST)
                    if replay is None:
                        sample = self.transforms(image=image, mask=mask)
                        replay = sample["replay"]
                    else:
                        sample = ReplayCompose.replay(replay, image=image, mask=mask)
                    image_ = sample["image"]
                    image_crops.append(image_)
                    mask_crops.append(sample["mask"])
                images = np.array(image_crops).astype(np.float32)
                masks = np.array(mask_crops).astype(np.float32)
                images = np.expand_dims(images, -1)
                masks = np.expand_dims(masks, -1)
                images = (images - image_mean) / image_std

                images = np.concatenate([images, images, masks], axis=-1)
                h = images.shape[0]
                if h > slice_size:
                    images = images[: slice_size]
                    all_images.append(np.moveaxis(images, -1, 0))
                    images = images[-slice_size:]
                    all_images.append(np.moveaxis(images, -1, 0))
                else:
                    if h != slice_size:
                        tmp = np.zeros((slice_size, *images.shape[1:]))
                        tmp[:h] = images
                        images = tmp
                    all_images.append(np.moveaxis(images, -1, 0))

        sample = {}
        sample['image'] = torch.from_numpy(np.array(all_images)).float()
        sample['label'] = torch.from_numpy(labels).float()
        sample['cube_id'] = cube_id
        return sample

    def __len__(self):
        return len(self.cases)

In [8]:

def load_checkpoint(model, checkpoint_path, strict=False, verbose=True):
    if verbose:
        print("=> loading checkpoint '{}'".format(checkpoint_path))
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    if 'state_dict' in checkpoint:
        state_dict = checkpoint['state_dict']
        state_dict = {re.sub("^module.", "", k): w for k, w in state_dict.items()}
        orig_state_dict = model.state_dict()
        mismatched_keys = []
        for k, v in state_dict.items():
            ori_size = orig_state_dict[k].size() if k in orig_state_dict else None
            if v.size() != ori_size:
                if verbose:
                    print("SKIPPING!!! Shape of {} changed from {} to {}".format(k, v.size(), ori_size))
                mismatched_keys.append(k)
        for k in mismatched_keys:
            del state_dict[k]
        model.load_state_dict(state_dict, strict=strict)
        del state_dict
        del orig_state_dict
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(checkpoint_path, checkpoint['epoch']))
    else:
        model.load_state_dict(checkpoint)
    del checkpoint


def load_model(conf: Dict, checkpoint: str):
    model = conf["network"](**conf["encoder_params"])
    model = model.cuda()
    load_checkpoint(model, checkpoint)
    return model.eval()

In [9]:
def process_segmentations(models: List[torch.nn.Module], test_dataset_dir: str, out_dir: str,  cases: List[str]):
    os.makedirs(out_dir, exist_ok=True)

    test_dataset = DatasetSeg(dataset_dir=test_dataset_dir, cases=cases)
    sampler = None
    oof_loader = DataLoader(
        test_dataset, batch_size=1, sampler=sampler, shuffle=False, num_workers=1, pin_memory=False
    )
    pred_dir = out_dir
    os.makedirs(pred_dir, exist_ok=True)
    for sample in tqdm(oof_loader):
        image = sample["image"]
        h = int(sample["h"][0])
        cube_id = sample["cube_id"][0]
        imgs = image.cpu().float()
        case_preds = np.zeros((imgs.shape[2], 256, 256), dtype=np.float32)

        with torch.no_grad():
            slices = get_slices(imgs, dim=2, window=256, overlap=128)
            for slice in slices:
                batch = imgs[:, :, slice.i_from:slice.i_to].cuda().float()
                with torch.cuda.amp.autocast(enabled=True):
                    preds = None
                    for model in models:
                        if preds is None:
                            preds = torch.softmax(model(batch)["mask"], dim=1)[0]
                        else:
                            preds += torch.softmax(model(batch)["mask"], dim=1)[0]
                    preds = torch.argmax(preds, dim=0)
                preds = preds.cpu().numpy()

                for pred_idx in range(slice.i_start, preds.shape[0]):
                    idx = slice.i_from + pred_idx
                    y_pred = preds[pred_idx]
                    case_preds[idx] = y_pred[:, :]
                torch.cuda.empty_cache()
        case_preds = np.array(case_preds)[:h]
        case_preds = case_preds.astype(np.uint8)
        tifffile.imwrite(os.path.join(pred_dir, f"{cube_id}.tif"), case_preds)


In [10]:
from torch import nn
def predict_classification(models: List[nn.Module], test_dataset_dir: str, cases: List, output_list: List):
    test_dataset = DatasetCrops(dataset_dir=test_dataset_dir, cases=cases)
    dataloader = DataLoader(
        test_dataset, batch_size=1, sampler=None, shuffle=False, num_workers=1, pin_memory=False
    )
    
    def predict_model(model):
        preds = []
        with torch.no_grad():
            for i in range(len(imgs)):
                with torch.cuda.amp.autocast():
                    output = model(imgs[i:i + 1])["cls"][0]
                pred_slice = torch.sigmoid(output.float()).cpu().numpy().astype(np.float32)
                with torch.cuda.amp.autocast():
                    output = model(torch.flip(imgs[i:i + 1], dims=(-1,)))["cls"][0]
                pred_slice += torch.sigmoid(output.float()).cpu().numpy().astype(np.float32)
                pred_slice /= 2
                preds.append(pred_slice)
        preds = np.max(np.array(preds), axis=0)
        preds[np.isnan(preds)] = 0.01
        return preds
        
    for sample in tqdm(dataloader):
        imgs = sample["image"].cuda().float()[0]
        cube_id = sample["cube_id"][0]
        with torch.no_grad():
            preds = []
            for model in models:
                preds.append(predict_model(model))
            preds = np.average(np.array(preds), axis=0)
            preds = np.clip(preds, 0.01, 0.99)
            output_list.append([cube_id, preds])

In [11]:
import zoo
config_seg  = {
  "network": zoo.ResNet3dCSN2P1D,
  "encoder_params": {
    "encoder": "r50ir"
  }
}
test_dataset_dir = "/kaggle/input/rsna-2022-cervical-spine-fracture-detection/test_images/"
cases = os.listdir(test_dataset_dir)
seg_model = load_model(config_seg, "/kaggle/input/rsna-weights/256_ResNet3dCSN2P1D_r50ir_0_dice")
ds = DatasetSeg(test_dataset_dir, cases)


=> loading checkpoint '/kaggle/input/rsna-weights/256_ResNet3dCSN2P1D_r50ir_0_dice'
=> loaded checkpoint '/kaggle/input/rsna-weights/256_ResNet3dCSN2P1D_r50ir_0_dice' (epoch 23)


In [12]:
preds_dir = "/kaggle/working/seg_preds"
process_segmentations([seg_model], test_dataset_dir,out_dir=preds_dir, cases=cases )

100%|██████████| 3/3 [00:13<00:00,  4.42s/it]


In [13]:
config_cls = {
  "network": zoo.ClassifierResNet3dCSN2P1D,
  "encoder_params": {
    "encoder": "r152ir",
    "num_classes": 8,
    "pool": "max"
  }
}
cls_models = [
    load_model(config_cls, "/kaggle/input/rsna-weights/swa_5_best_full_ClassifierResNet3dCSN2P1D_r152ir_0.pth"),
    load_model(config_cls, "/kaggle/input/rsna-weights/swa_5_best_full_ClassifierResNet3dCSN2P1D_r152ir_1.pth"),
    load_model(config_cls, "/kaggle/input/rsna-weights/swa_5_best_full_ClassifierResNet3dCSN2P1D_r152ir_2.pth"),
    load_model(config_cls, "/kaggle/input/rsna-weights/swa_5_best_full_ClassifierResNet3dCSN2P1D_r152ir_3.pth"),

]

config_cls = {
  "network": zoo.ClassifierResNet3dCSN2P1D,
  "encoder_params": {
    "encoder": "r152ip",
    "num_classes": 8,
    "pool": "max"
  }
}

cls_models += [
    load_model(config_cls, "/kaggle/input/rsna-weights/swa_3_best_full_frozen_ClassifierResNet3dCSN2P1D_r152ip_0.pth"),
    load_model(config_cls, "/kaggle/input/rsna-weights/swa_3_best_full_frozen_ClassifierResNet3dCSN2P1D_r152ip_1.pth"),
    load_model(config_cls, "/kaggle/input/rsna-weights/swa_3_best_full_frozen_ClassifierResNet3dCSN2P1D_r152ip_2.pth"),
    load_model(config_cls, "/kaggle/input/rsna-weights/swa_3_best_full_frozen_ClassifierResNet3dCSN2P1D_r152ip_3.pth"),

]

=> loading checkpoint '/kaggle/input/rsna-weights/swa_5_best_full_ClassifierResNet3dCSN2P1D_r152ir_0.pth'
=> loaded checkpoint '/kaggle/input/rsna-weights/swa_5_best_full_ClassifierResNet3dCSN2P1D_r152ir_0.pth' (epoch 0)
=> loading checkpoint '/kaggle/input/rsna-weights/swa_5_best_full_ClassifierResNet3dCSN2P1D_r152ir_1.pth'
=> loaded checkpoint '/kaggle/input/rsna-weights/swa_5_best_full_ClassifierResNet3dCSN2P1D_r152ir_1.pth' (epoch 0)
=> loading checkpoint '/kaggle/input/rsna-weights/swa_5_best_full_ClassifierResNet3dCSN2P1D_r152ir_2.pth'
=> loaded checkpoint '/kaggle/input/rsna-weights/swa_5_best_full_ClassifierResNet3dCSN2P1D_r152ir_2.pth' (epoch 0)
=> loading checkpoint '/kaggle/input/rsna-weights/swa_5_best_full_ClassifierResNet3dCSN2P1D_r152ir_3.pth'
=> loaded checkpoint '/kaggle/input/rsna-weights/swa_5_best_full_ClassifierResNet3dCSN2P1D_r152ir_3.pth' (epoch 0)
=> loading checkpoint '/kaggle/input/rsna-weights/swa_3_best_full_frozen_ClassifierResNet3dCSN2P1D_r152ip_0.pth'
=> 

In [14]:
output = []

predict_classification(cls_models, test_dataset_dir, cases, output)

100%|██████████| 3/3 [00:35<00:00, 11.83s/it]


In [15]:
import pandas as pd
sub_df = pd.read_csv('/kaggle/input/rsna-2022-cervical-spine-fracture-detection/test.csv')

del sub_df['prediction_type']
del sub_df['StudyInstanceUID']

data = []
for cube_id, preds in output:
    data.append([f"{cube_id}_patient_overall", preds[0]])
    for i in range(1, 8):
        data.append([f"{cube_id}_C{i}", preds[i]])
pred_df = pd.DataFrame(data, columns=["row_id", "fractured"])
sub_df = sub_df.merge(pred_df, on=['row_id'])
sub_df.to_csv('submission.csv',index=False)
sub_df.head(50)

,row_id,fractured


In [16]:
import shutil
shutil.rmtree('/kaggle/working/seg_preds')